<a href="https://colab.research.google.com/github/Santosdevbjj/estoqAWSsagemakerCanvas/blob/main/notebooks/01_data_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Geração do Dataset Sintético de Previsão de Estoque

Este notebook é responsável por criar o arquivo de dados `estoque_historico_sazonal.csv` para ser utilizado no treinamento do modelo de Machine Learning no Amazon SageMaker Canvas. O dataset simula 3 anos de vendas diárias e inclui componentes de sazonalidade e ruído, cruciais para um modelo de previsão robusto.

In [4]:

# Importação das bibliotecas necessárias
import pandas as pd
import numpy as np
from datetime import timedelta

# Define a estrutura de pastas para salvar o arquivo (garantindo que existe)
CAMINHO_PASTA = 'datasets/raw/'
NOME_ARQUIVO = 'estoque_historico_sazonal.csv'

import os
# A linha abaixo é importante se você estiver executando o notebook no Colab ou localmente
os.makedirs(CAMINHO_PASTA, exist_ok=True)

print(f"Pasta de destino criada: {CAMINHO_PASTA}")

Pasta de destino criada: datasets/raw/


In [5]:

import pandas as pd
import numpy as np
from datetime import timedelta

# Configurações do Dataset
N_DIAS = 1095  # 3 anos de dados (365 * 3)
DATA_INICIO = pd.to_datetime('2023-01-01')
ID_PRODUTO = 'PROD_A'

# 1. Criação do Index de Datas
datas = [DATA_INICIO + timedelta(days=i) for i in range(N_DIAS)]

# 2. Geração da Base de Vendas (Vendas_Historicas_Unidades)
vendas_base = 150 + (np.sin(np.linspace(0, 6 * np.pi, N_DIAS)) * 50) # Tendência sinusoidal (sazonalidade)
ruido_aleatorio = np.random.normal(0, 20, N_DIAS)
vendas_historicas = (vendas_base + ruido_aleatorio).astype(int)
vendas_historicas = np.maximum(50, vendas_historicas) # Garante que as vendas sejam positivas

# 3. Adição de Picos Sazonais (Natal e Black Friday/Meio do Ano)
for i, data in enumerate(datas):
    mes = data.month
    dia = data.day
    # Pico de Natal (Dezembro)
    if mes == 12:
        vendas_historicas[i] += np.random.randint(50, 150)
    # Pico de Black Friday/Meio do Ano (Junho e Novembro)
    if mes in [6, 11]:
        vendas_historicas[i] += np.random.randint(30, 100)
    # Finais de semana (vendas maiores)
    if data.weekday() >= 5: # Sábado (5) ou Domingo (6)
        vendas_historicas[i] += np.random.randint(10, 30)

# 4. Geração de Outras Colunas
promocao_ativa = np.random.choice([True, False], size=N_DIAS, p=[0.20, 0.80])
preco_unitario = np.random.uniform(18.0, 22.0, N_DIAS).round(2)
lead_time = np.random.randint(5, 10, N_DIAS)
estoque_atual = np.cumsum(-vendas_historicas) + 15000 # Estoque baseado na venda acumulada e reabastecimentos simulados
estoque_atual[estoque_atual < 0] = np.random.randint(500, 1000) # Simula um reabastecimento antes de zerar
estoque_atual = estoque_atual + np.random.randint(500, 2000, N_DIAS) # Reabastecimento aleatório
estoque_atual = estoque_atual % 5000 + 500 # Mantém o estoque em uma faixa razoável

# Previsão de Demanda (Target: Vendas dos próximos 30 dias)
# Simplificamos o alvo como sendo as vendas históricas do dia com um pequeno offset e ruído
previsao_demanda_30d = (vendas_historicas * 1.1 + np.random.normal(0, 10, N_DIAS)).astype(int)
previsao_demanda_30d = np.maximum(80, previsao_demanda_30d)

# 5. Criação do DataFrame
df = pd.DataFrame({
    'Data': datas,
    'ID_Produto': ID_PRODUTO,
    'Preco_Unitario': preco_unitario,
    'Promocao_Ativa': promocao_ativa,
    'Lead_Time_Dias': lead_time,
    'Vendas_Historicas_Unidades': vendas_historicas,
    'Estoque_Atual': estoque_atual.astype(int),
    'Previsao_Demanda_30D': previsao_demanda_30d
})

# 6. Exportação do Arquivo CSV
NOME_ARQUIVO = 'estoque_historico_sazonal.csv'
CAMINHO_PASTA = 'datasets/raw/'

# Cria o diretório se não existir
import os
os.makedirs(CAMINHO_PASTA, exist_ok=True)

df.to_csv(CAMINHO_PASTA + NOME_ARQUIVO, index=False)

print(f"Dataset criado com sucesso!")
print(f"Total de Linhas: {len(df)}")
print(f"Caminho no GitHub: {CAMINHO_PASTA}{NOME_ARQUIVO}")
print("\nPrimeiras 5 linhas:")
print(df.head())

Dataset criado com sucesso!
Total de Linhas: 1095
Caminho no GitHub: datasets/raw/estoque_historico_sazonal.csv

Primeiras 5 linhas:
        Data ID_Produto  Preco_Unitario  Promocao_Ativa  Lead_Time_Dias  \
0 2023-01-01     PROD_A           20.27           False               7   
1 2023-01-02     PROD_A           21.58           False               9   
2 2023-01-03     PROD_A           19.45           False               6   
3 2023-01-04     PROD_A           19.88           False               9   
4 2023-01-05     PROD_A           19.76           False               6   

   Vendas_Historicas_Unidades  Estoque_Atual  Previsao_Demanda_30D  
0                         140           1989                   169  
1                         131           1302                   130  
2                         143           1760                   151  
3                         153           1466                   152  
4                         138            527                   170  


In [6]:

# Exportação do Arquivo CSV
df.to_csv(CAMINHO_PASTA + NOME_ARQUIVO, index=False)

print(f"Dataset '{NOME_ARQUIVO}' criado com sucesso!")
print(f"Total de Linhas: {len(df)}")

# Verificação das primeiras linhas
print("\nVerificação das Primeiras 5 Linhas do CSV Final:")
print(df.head())

Dataset 'estoque_historico_sazonal.csv' criado com sucesso!
Total de Linhas: 1095

Verificação das Primeiras 5 Linhas do CSV Final:
        Data ID_Produto  Preco_Unitario  Promocao_Ativa  Lead_Time_Dias  \
0 2023-01-01     PROD_A           20.27           False               7   
1 2023-01-02     PROD_A           21.58           False               9   
2 2023-01-03     PROD_A           19.45           False               6   
3 2023-01-04     PROD_A           19.88           False               9   
4 2023-01-05     PROD_A           19.76           False               6   

   Vendas_Historicas_Unidades  Estoque_Atual  Previsao_Demanda_30D  
0                         140           1989                   169  
1                         131           1302                   130  
2                         143           1760                   151  
3                         153           1466                   152  
4                         138            527                   170  
